# 📓 Course Notebook — From Python Basics to Industrial Data Preprocessing

Welcome! This notebook is designed to **evolve step by step** with the course.  
Instead of starting fresh each time, we will **gradually extend this notebook** as new topics are introduced.

### 🔹 Current Scope
- **Chapter 2 — Python Basics & Exploratory Signal Analysis**  
  Learn Python essentials for scientific computing, plotting, and working with real industrial datasets.  
  Explore signals in the **time domain, frequency domain, and using rolling statistics**.  
  Automate feature extraction and visualization.  

- **Chapter 3 — Data Preprocessing**  
  Prepare raw signals for machine learning.  
  Cover techniques such as **handling missing values, outlier detection, noise filtering, scaling, transformations, and feature extraction**.  
  Practice splitting data into training, validation, and test subsets.

### 🔹 Upcoming Chapters
- **Chapter 4 — Classical Machine Learning**  
  Build and evaluate models using **supervised and unsupervised ML techniques** (e.g., classification, clustering, dimensionality reduction).  
  Apply cross-validation, feature selection, and model comparison.  

- **Chapter 5 — Deep Learning for Industrial Data**  
  Design and train **neural networks** for time-series and signal data.  
  Explore architectures such as CNNs, RNNs, and Transformers, and compare them with classical ML methods.

### 🔹 Your Parallel Project
Alongside the Paderborn bearing dataset used in this notebook, each student must choose **one industrial dataset** at the beginning of the course.  
At the end of each chapter, apply the learned concepts and preprocessing steps on your **own dataset**, ensuring it is ready for the next chapter.  
This step-by-step extension will become your **final course project**.

### 🔹 Exercises
- Each chapter ends with a **dedicated exercise section**.  
- Exercises are practical and progressive — you are expected to **complete all exercises** at the end of each chapter.  
- Some tasks are marked as *optional* for deeper exploration.  

---

✅ By the end of Chapter 3, you should be comfortable with:  
- Using Python for industrial data analysis.  
- Exploring, cleaning, and transforming raw signals.  
- Preparing datasets (Paderborn + your own) for **feature extraction and machine learning** in the next chapter.

# 📘 **Chapter 2: Python for Machine Learning**
---

This chapter introduces Python tools for working with industrial datasets.  
Skills here (data loading, visualization, statistics) are foundations for later chapters on ML (Ch. 4) and DL (Ch. 5).  

**Datasets used:** Paderborn (main example).

In [ ]:
# ---- Imports ----

import os
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
from urllib.parse import urljoin
import ssl
import shutil
import platform
import subprocess

# Ensure inline plotting
%matplotlib inline

try:
    import rarfile
except ImportError:
    !pip install rarfile
    import rarfile

In [ ]:
# ---- Dataset Source Configuration ----

# Detect environment
try:
    import google.colab
    ON_COLAB = True
except ImportError:
    ON_COLAB = False

if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COURSE_PATH = "/content/drive/MyDrive/Industrial_ML_Course"
else:
    COURSE_PATH = r"D:\Industrial_ML_Course"  # adjust if needed

DATASET_PATH = os.path.join(COURSE_PATH, "datasets/Paderborn")
NOTEBOOK_PATH = os.path.join(COURSE_PATH, "notebooks")

os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(NOTEBOOK_PATH, exist_ok=True)

print("Environment:", "Colab" if ON_COLAB else "Local")
print("Dataset Path:", DATASET_PATH)
print("Notebook Path:", NOTEBOOK_PATH)

In [ ]:
# ---- Download and Extract Dataset ----

sample_file = 'K001.rar'  # example file
dataset_url = 'https://groups.uni-paderborn.de/kat/BearingDataCenter/'  # official hosting
file_url = urljoin(dataset_url, sample_file)
file_path = os.path.join(DATASET_PATH, sample_file)
extracted_folder = os.path.splitext(file_path)[0]

ssl_context = ssl._create_unverified_context()

if not os.path.exists(extracted_folder):
    if not os.path.exists(file_path):
        print(f"Downloading {sample_file} ...")
        with urllib.request.urlopen(file_url, context=ssl_context) as response, open(file_path, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
        print("Download completed!")
    else:
        print(f"{sample_file} already exists.")

    print(f"Extracting {sample_file} ...")
    try:
        with rarfile.RarFile(file_path) as rf:
            rf.extractall(DATASET_PATH)
    except rarfile.Error:
        print("rarfile failed, trying WinRAR...")
        winrar_path = r"C:\Program Files\WinRAR\WinRAR.exe"  # adjust if needed
        subprocess.run([winrar_path, "x", "-o+", file_path, DATASET_PATH], check=True)
    print("Extraction completed!")

    # Optional cleanup (commented out for inspection)
    # if os.path.exists(file_path):
    #     os.remove(file_path)
    #     print(f"{sample_file} removed after extraction.")
else:
    print(f"Extracted folder already exists: {extracted_folder}")

# List all .mat files
mat_files = []
for root, dirs, files in os.walk(extracted_folder):
    for f in files:
        if f.endswith(".mat"):
            mat_files.append(os.path.join(root, f))

print(f"Found {len(mat_files)} .mat files. Examples:", mat_files[:5])

In [ ]:
# ---- Load a specific .mat file ----

mat_file = mat_files[0]  # first file
print("Loading:", mat_file)

data = sio.loadmat(mat_file, squeeze_me=True, struct_as_record=False)

keys = [k for k in data.keys() if not k.startswith('__')]
print("Measurement keys:", keys)

measurement = data[keys[0]]  # usually one measurement per file
print("Available fields:", measurement._fieldnames)

In [ ]:
# ---- Access 'Info' field ----

info = measurement.Info
if hasattr(info, '__dict__'):
    print("Info fields:", vars(info).keys())
    for k, v in vars(info).items():
        print(f"{k}: {type(v)} -> {v}")
else:
    print("Info:", info)

In [ ]:
# ---- Access Y (sensor recordings) ----

Y = measurement.Y

print("Available signals:")
if isinstance(Y, np.ndarray):
    for i, sig in enumerate(Y.flat):
        fs = getattr(sig, "Raster", "N/A")
        shape = getattr(sig, "Data", np.array([])).shape
        length = len(sig.Data) if hasattr(sig, "Data") else 0
        print(f"{i+1:2d}. {sig.Name:25s} | fs={fs:<10} | shape={shape} | length={length}")
else:
    fs = getattr(Y, "Raster", "N/A")
    shape = getattr(Y, "Data", np.array([])).shape
    length = len(Y.Data) if hasattr(Y, "Data") else 0
    print(f" 1. {Y.Name:25s} | fs={fs:<10} | shape={shape} | length={length}")

In [ ]:
# ---- Select and visualize a signal ----

signal_index = 1
max_samples_to_plot = 5000

sig = Y.flat[signal_index] if isinstance(Y, np.ndarray) else Y
fs = sig.Raster if hasattr(sig, "Raster") and isinstance(sig.Raster, (int,float)) else 64000
data_to_plot = sig.Data[:max_samples_to_plot]

print(f"Selected Signal: {sig.Name}")
print(f"Sampling frequency: {fs} Hz")
print(f"Data shape: {sig.Data.shape}")

# Time-domain plot
plt.figure(figsize=(12,4))
plt.plot(data_to_plot)
plt.title(f"Signal {signal_index+1}: {sig.Name} (first {len(data_to_plot)} samples)")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ---- Rolling Statistics ----

signal_series = pd.Series(data_to_plot)
rolling_mean = signal_series.rolling(window=200).mean()
rolling_std = signal_series.rolling(window=200).std()
rolling_rms = signal_series.rolling(window=200).apply(lambda x: np.sqrt(np.mean(x**2)))

print("Rolling mean (first 5):", rolling_mean.dropna().head().values)

plt.figure(figsize=(12,4))
plt.plot(signal_series, label="Original", alpha=0.7)
plt.plot(rolling_mean, label="Rolling Mean", linewidth=2)
plt.plot(rolling_std, label="Rolling Std", linewidth=2)
plt.plot(rolling_rms, label="Rolling RMS", linewidth=2)
plt.title(f"Rolling Statistics - {sig.Name}")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ---- Frequency-domain analysis ----

from scipy.fft import fft, fftfreq

N = len(data_to_plot)
T = 1/fs
yf = fft(data_to_plot)
xf = fftfreq(N, T)[:N//2]

plt.figure(figsize=(12,4))
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.title(f"Frequency Spectrum - {sig.Name}")
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude")
plt.xlim(0, fs/2)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ---- Spectrogram (Time-Frequency) ----

from scipy.signal import spectrogram

f, t, Sxx = spectrogram(data_to_plot, fs=fs)
plt.figure(figsize=(12,4))
plt.pcolormesh(t, f, 10*np.log10(Sxx), shading='gouraud')
plt.title(f"Spectrogram - {sig.Name}")
plt.ylabel("Frequency [Hz]")
plt.xlabel("Time [s]")
plt.colorbar(label="Power [dB]")
plt.ylim(0, fs/2)
plt.tight_layout()
plt.show()

In [ ]:
# ---- Optional: Batch load all signals ----

def load_all_signals(mat_files):
    signals = []
    for mat_file in mat_files:
        data = sio.loadmat(mat_file, squeeze_me=True, struct_as_record=False)
        measurement_key = [k for k in data.keys() if not k.startswith('__')][0]
        measurement = data[measurement_key]
        Y = measurement.Y
        if isinstance(Y, np.ndarray):
            for sig in Y.flat:
                signals.append(sig.Data)
        else:
            signals.append(Y.Data)
    return signals

all_signals = load_all_signals(mat_files)
print(f"Total signals loaded: {len(all_signals)}")

## 📝 Exercises — Chapter 2: Python Basics

These exercises are meant to reinforce **Python programming, plotting, and exploratory data handling**. You will work both on the Paderborn dataset and your own chosen industrial dataset (your “project dataset”).

1. **Signal Comparison**
   - Load two signals from different conditions (healthy vs faulty).
   - Plot them in:
     - **Time domain**
     - **Frequency domain (FFT)**
     - **Rolling statistics** (mean, RMS, std)
   - Observe differences in amplitude and patterns.
   - Take into account that different signals may have **different sampling frequencies or lengths**.

2. **Rolling Statistics**
   - Experiment with different **window sizes** for rolling mean, RMS, or standard deviation.
   - How does smoothing affect visibility of patterns or trends?

3. **Spectrogram Analysis**
   - Generate **spectrograms** for multiple signals.
   - Compare energy distributions for healthy vs faulty conditions.
   - Identify patterns that may indicate faults.

4. **Automation Challenge**
   - Write a function to **load multiple `.mat` files** and extract a **feature matrix**.
   - Features to extract (time-domain):
     - mean, std, RMS, peak-to-peak, skewness, kurtosis
   - If signals are long, divide them into **non-overlapping frames/windows** before feature extraction.
   - Visualize relationships between extracted features using scatter plots or heatmaps.
   - Do certain features form **distinct clusters** for healthy vs faulty bearings?  
     *Hint:* If features cluster differently, this suggests they can help distinguish healthy vs faulty conditions. You can **optionally** use dimensionality reduction (PCA, t-SNE) to visualize clustering in 2D/3D.

5. **Project Dataset Exploration**
   - Repeat all experiments and explorations from this notebook (plotting, FFT, rolling statistics, etc.) on your chosen industrial dataset.
   - Make sure your dataset is **ready for preprocessing in Chapter 3**.

# 📘 **Chapter 3: Data Preprocessing**
---

This chapter extends our Paderborn dataset exploration to cover:

1. **Data Cleaning**  
   - Removing Duplicates  
   - Handling Missing Values  
   - Outlier Detection & Correction  
   - Noise Filtering  

2. **Data Transformation**  
   - Scaling (Normalization & Standardization)  
   - Distribution Transforms (Power, Log)  
   - Categorical Encoding (Label & One-hot)  

3. **Feature Extraction**  

4. **Data Splitting & Validation**  

In [ ]:
# ---- Duplicate Detection (Meaningful for Time-Amplitude Pairs) ----
'''
⚡ Key Point:
For raw 1D signals, repeated amplitude values are natural (e.g., sine waves)
and do NOT indicate duplication.

In time-series data, however, a *duplicate sample* means the same
(time, amplitude) pair is logged more than once — usually due to
logging or communication errors. These duplicates should be removed.
'''

# Represent the signal as a time–amplitude DataFrame (so we can check duplicates properly)
time_index = np.arange(len(signal_series))
signal_df = pd.DataFrame({"time": time_index, "amplitude": signal_series})

# ✅ Simulate duplicates by repeating the first 200 rows
duplicated_df = pd.concat([signal_df, signal_df.iloc[:200]], ignore_index=True)

print("Original length:", len(signal_df))
print("With duplicates:", len(duplicated_df))
print("Duplicates present?", duplicated_df.duplicated().any())

# Show a few duplicated rows for illustration
print("\nExample duplicated samples:")
print(duplicated_df[duplicated_df.duplicated()].head())

# Remove duplicates (drops rows where *both* time and amplitude are repeated)
clean_df = duplicated_df.drop_duplicates()
print("\nAfter removing duplicates:", len(clean_df))
print("Number of rows removed:", len(duplicated_df) - len(clean_df))

In [ ]:
# ---- Adding Missing Values (Simulated) ---
'''
The Paderborn dataset we are using is clean — there are no missing samples.
However, in real-world industrial systems, sensors sometimes fail to record data points
due to communication errors, noise, or temporary malfunctions.
To practice, let’s **simulate missing values** by randomly dropping some samples,
and then apply the techniques you learned earlier to handle them.
'''

# Convert signal into a pandas Series (easier to manipulate)
signal_series = pd.Series(data_to_plot.copy())

# Introduce missing values (simulate 10% of samples as NaN)
rng = np.random.default_rng(seed=42)
missing_indices = rng.choice(signal_series.index, size=int(0.1 * len(signal_series)), replace=False)
signal_series.loc[missing_indices] = np.nan

print("Number of missing values:", signal_series.isna().sum())

# Plot signal with missing values
plt.figure(figsize=(12, 3))
plt.subplot(2,1,1)
plt.plot(signal_series, color='red', alpha=0.7)
plt.title("Signal with Simulated Missing Values")
plt.ylabel("Amplitude")
plt.subplot(2,1,2) # Zoom in to better visualize missing gaps
plt.plot(signal_series[:500], color='red', alpha=0.7)
plt.title("Zoomed-In Signal with Simulated Missing Values")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
# --- Handling Missing Values ----
'''
Strategies to Handle Missing Values in Time Series:

1. Drop missing values – risky: creates irregular gaps in time, breaking continuity.
2. Forward/Backward fill – simple; assumes stability of signal over gaps.
3. Interpolation – often preferred; estimates values smoothly between neighbors.

👉 In industrial signals, interpolation is usually more realistic than filling
with constant values or dropping data.
'''

# Forward fill
ffill_signal = signal_series.fillna(method="ffill")

# Backward fill
bfill_signal = signal_series.fillna(method="bfill")

# Linear interpolation
interp_signal = signal_series.interpolate(method="linear")

# Plot comparison
plt.figure(figsize=(12, 4))
plt.subplot(211)
plt.plot(signal_series, label="Original (with NaN)", color="red", alpha=0.5)
plt.plot(interp_signal, label="Interpolated", color="blue")
plt.legend()
plt.title("Handling Missing Values in the Signal")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.subplot(212) # Zoom in to better visualize missing gaps
plt.plot(signal_series[:500], label="Original (with NaN)", color="red", alpha=0.5)
plt.plot(interp_signal[:500], label="Interpolated", color="blue")
plt.legend()
plt.title("Handling Missing Values in the Signal (Zoomed-In)")
plt.xlabel("Sample index")
plt.ylabel("Amplitude")
plt.show()

⚠️ **Note on Handling Missing Values in Signals**

- Industrial signals are often continuous, so **missing samples** usually come from sensor faults, communication errors, or data loss.  
- Simply **dropping NaNs** can break the time series and distort analysis.  
- Better strategies:  
  - **Forward/Backward fill**: copies last/next valid sample (good for short gaps).  
  - **Interpolation**: estimates values smoothly between neighbors (better for gradual changes).  
- The choice depends on the **context**: for vibration signals, interpolation often works well; for status signals (on/off), forward fill may be more realistic.  

In [ ]:
# ---- Outlier Detection (Z-score method) ----

signal_series = pd.Series(data_to_plot)

z_scores = (signal_series - signal_series.mean()) / signal_series.std()
outliers = np.where(np.abs(z_scores) > 3)[0]
print(f"Detected {len(outliers)} outliers")

plt.figure(figsize=(12,3))
plt.plot(signal_series, label="Signal")
plt.scatter(outliers, signal_series.iloc[outliers], color='red', label="Outliers")
plt.legend()
plt.show()

⚠️ **Note on Outlier Detection in Signals**

- Outliers are samples that deviate strongly from the normal signal range.  
- Common sources in industrial data:  
  - Sensor glitches or random noise spikes  
  - Short-term machine/system anomalies  
- The **Z-score method** marks values far from the mean (e.g., > 3 standard deviations).  
- Interpretation is key:  
  - Outliers may be **true faults** or just **measurement errors**.  
  - Decide whether to remove, correct, or keep them using domain knowledge.  
- In 1D signals, small isolated spikes are common — don’t misclassify normal variations as outliers.  

In [ ]:
# ---- Noise Filtering (Median Filter) ----
'''
Median filtering is a simple and robust method to reduce short-term spikes/noise
while preserving the overall shape of the signal.

- Why median? Unlike mean filters, it is less sensitive to outliers and sharp spikes.
- Here we apply it with kernel_size=5 (uses 5-sample sliding window).
- We fill NaNs beforehand (forward-fill) to avoid issues during filtering.
'''

from scipy.signal import medfilt

# Fill NaNs before filtering
signal_filled = signal_series.fillna(method="ffill")

# Apply median filter
filtered_signal = medfilt(signal_filled, kernel_size=5)

# Plot comparison
plt.figure(figsize=(12,3))
plt.plot(signal_filled, alpha=0.6, label="Original (filled)")
plt.plot(filtered_signal, label="Median Filtered", color="orange")
plt.legend()
plt.title("Noise Filtering with Median Filter")
plt.show()

⚠️ **Note on Noise Filtering in Signals**

- Industrial signals often contain **high-frequency noise** from sensors or electrical interference.  
- **Median filtering** reduces noise while **preserving edges and sharp transitions**.  
- How it works:  
  - Each sample is replaced by the **median** of neighboring values within a window.  
  - Especially effective for removing **impulse spikes** or isolated noise points.  
- Interpretation:  
  - Smoothed signals reveal underlying trends more clearly.  
  - Window size matters: too large → removes meaningful variations; too small → noise remains.  

In [ ]:
# ---- Scaling (Normalization & Standardization) ----

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Convert to numpy 2D (required by sklearn)
X = signal_series.fillna(method="ffill").values.reshape(-1, 1)

# Normalization (Min-Max)
scaler_minmax = MinMaxScaler()
X_minmax = scaler_minmax.fit_transform(X)

# Standardization (Z-score)
scaler_std = StandardScaler()
X_std = scaler_std.fit_transform(X)

# Plot histograms
plt.figure(figsize=(12, 4))
plt.hist(X, bins=50, alpha=0.5, label="Original", density=True)
plt.hist(X_minmax, bins=50, alpha=0.5, label="Min-Max", density=True)
plt.hist(X_std, bins=50, alpha=0.5, label="Standardized", density=True)
plt.legend()
plt.title("Distribution After Scaling (Normalization & Standardization)")
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

In [ ]:
# ---- Distribution Transforms (Power & Log) ----

from sklearn.preprocessing import PowerTransformer

# Convert to numpy 2D (required by sklearn)
X = signal_series.fillna(method="ffill").values.reshape(-1, 1)

# Power transform (Yeo-Johnson, handles zeros/negatives)
pt = PowerTransformer(method='yeo-johnson')
X_pt = pt.fit_transform(X)

# Plot histograms
plt.figure(figsize=(12, 4))
plt.hist(X, bins=50, alpha=0.5, label="Original", density=True)
plt.hist(X_pt, bins=50, alpha=0.5, label="Power Transform (Yeo-Johnson)", density=True)
plt.legend()
plt.title("Distribution After Distribution Transforms")
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

In [ ]:
# ---- Scaling & Transformations ----

from sklearn.preprocessing import PowerTransformer

# Convert to numpy 2D (required by sklearn)
X = signal_series.fillna(method="ffill").values.reshape(-1, 1)

# Normalization (Min-Max)
scaler_minmax = MinMaxScaler()
X_minmax = scaler_minmax.fit_transform(X)

# Standardization (Z-score)
scaler_std = StandardScaler()
X_std = scaler_std.fit_transform(X)

# Power transform (Yeo-Johnson, handles zeros/negatives)
pt = PowerTransformer(method='yeo-johnson')
X_pt = pt.fit_transform(X)

# Plot histograms
plt.figure(figsize=(12, 4))
plt.hist(X, bins=50, alpha=0.5, label="Original", density=True)
plt.hist(X_minmax, bins=50, alpha=0.5, label="Min-Max", density=True)
plt.hist(X_std, bins=50, alpha=0.5, label="Standardized", density=True)
plt.hist(X_pt, bins=50, alpha=0.5, label="Power Transform", density=True)
plt.legend()
plt.title("Distribution After Different Transformations")
plt.xlabel("Value")
plt.ylabel("Density")
plt.show()

⚠️ **Notes on Scaling & Transformations**

**🔹 What we are doing**
- Scaling and transformations make features more suitable for machine learning.  
- In this example, we compare:  
  - **Original signal distribution** (nearly sinusoidal current signal).  
  - **Min–Max scaling** → squeezes values into `[0, 1]`.  
  - **Standardization (z-score)** → mean = 0, variance = 1.  
  - **Power Transform (Yeo–Johnson)** → reduces skewness and makes distributions closer to Gaussian.  



**🔹 Observations from the plots**
- **Original distribution**: Two peaks appear because the sine wave spends more time near its extrema (± amplitude). This is expected and not “true bimodality.”  
- **Min–Max scaling**: Same shape, but mapped into `[0, 1]`.  
- **Standardization**: Same shape, but shifted and rescaled to mean 0, variance 1.  
- **Power Transform**: Looks almost identical to Standardization here, because the signal is already symmetric (no skewness to correct).  

**Important:** Power Transform mainly helps when data is **skewed with long tails**. For symmetric oscillatory signals like these, it provides little benefit over standardization.  


**🔹 Why this matters**
- Distance-based ML algorithms (**SVMs, kNN, neural networks**) need scaled inputs.  
- Algorithms assuming Gaussian features (**PCA, linear regression**) may benefit from transformations if skewness exists.  
- For sinusoidal signals, Standardization is usually sufficient.  


In [ ]:
# ---- Feature Extraction ----

from scipy.stats import kurtosis, skew

def extract_features(sig, fs, band=(1000, 5000)):
    """
    Extract time-domain and frequency-domain features from a 1D signal.

    Parameters:
        sig (array-like): Input signal
        fs (float): Sampling frequency (Hz)
        band (tuple): Frequency range for band power calculation (Hz)

    Returns:
        features (dict): Dictionary of extracted features
    """
    sig = np.ravel(sig)  # ensure 1D
    if len(sig) == 0:
        # handle empty signal gracefully
        return {key: np.nan for key in ["mean","std","rms","peak2peak",
                                        "skewness","kurtosis","dominant_freq","band_power"]}

    # Time-domain features
    features = {
        "mean": np.mean(sig),
        "std": np.std(sig),
        "rms": np.sqrt(np.mean(sig**2)),
        "peak2peak": np.ptp(sig),
        "skewness": skew(sig),
        "kurtosis": kurtosis(sig),
    }

    # Frequency-domain features
    yf = np.abs(fft(sig))[:len(sig)//2]      # magnitude spectrum
    xf = fftfreq(len(sig), 1/fs)[:len(sig)//2]  # corresponding frequencies

    features["dominant_freq"] = xf[np.argmax(yf)]
    features["band_power"] = np.sum(yf[(xf > band[0]) & (xf < band[1])])

    return features

# Example usage
feat = extract_features(signal_series.fillna(method="ffill"), fs)
display("Extracted Features:", feat)

## 🛠 Feature Extraction

### 🔹 What we are doing
- Extracting **numerical descriptors** from signals that summarize their characteristics.
- Two main types of features:
  1. **Time-domain features:** mean, std, RMS, peak-to-peak, skewness, kurtosis.
  2. **Frequency-domain features:** dominant frequency, band power.

---

### 🔹 Why it matters
- Raw signals are often high-dimensional and redundant; features reduce dimensionality while keeping essential information.
- Time-domain features capture **overall amplitude, variability, and shape**.
- Frequency-domain features capture **periodicities and energy in specific frequency bands**.

---

### 🔹 Interpretation
- **Mean:** average signal amplitude.
- **Std:** variability of the signal.
- **RMS:** overall energy magnitude.
- **Peak-to-peak:** amplitude range.
- **Skewness:** asymmetry of signal distribution.
- **Kurtosis:** presence of heavy tails or sharp peaks.
- **Dominant frequency:** main periodic component.
- **Band power:** energy in a specific frequency range.

In [ ]:
# ---- Windowed Feature Extraction (Non-overlapping) ----

window_length = 5000  # number of samples per frame
feature_matrix = []
labels = []

for i, mat_file in enumerate(mat_files[:20]):  # demo subset
    data = sio.loadmat(mat_file, squeeze_me=True, struct_as_record=False)
    measurement_key = [k for k in data.keys() if not k.startswith('__')][0]
    measurement = data[measurement_key]

    # Extract raw signal data
    sig = measurement.Y.flat[0] if isinstance(measurement.Y, np.ndarray) else measurement.Y
    sig_data = sig.Data if hasattr(sig, "Data") else np.array(sig)
    fs = sig.Raster if hasattr(sig, "Raster") and isinstance(sig.Raster, (int,float)) else 64000

    # Slide through the signal in non-overlapping windows
    num_frames = len(sig_data) // window_length
    for j in range(num_frames):
        frame = sig_data[j*window_length:(j+1)*window_length]
        feat = extract_features(frame, fs)
        feature_matrix.append(list(feat.values()))
        labels.append(measurement.Label if hasattr(measurement, "Label") else "Unknown")

feature_matrix = np.array(feature_matrix)
labels = np.array(labels)

print(f'Feature Matrix Size: {feature_matrix.shape}\nLabels Size: {labels.shape}')

In [ ]:
# Train / Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    feature_matrix, labels, test_size=0.3, random_state=42, stratify=labels
)

print("Train / Test Split:")
print("Train size:", X_train.shape, "Test size:", X_test.shape)

In [ ]:
# Train / Validation / Test Split

# First split into train+val and test
X_temp, X_test, y_temp, y_test = train_test_split(
    feature_matrix, labels, test_size=0.2, stratify=labels, random_state=42
)

# Then split train into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)  # 0.25 x 0.8 = 0.2
print("Train / Validation / Test Split:")
print("Train size:", X_train.shape, " Validation size:", X_val.shape, " Test size:", X_test.shape)

In [ ]:
# ---- K-Fold Cross-Validation ----
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=5, shuffle=True, random_state=42)

clf = RandomForestClassifier()
scores = cross_val_score(clf, feature_matrix, labels, cv=kf)

print("Cross-validation scores:", scores)
print("Mean CV score:", scores.mean())

## 📊 Data Splitting & Validation

- Before training ML models, we must **evaluate performance reliably**.  
- The goal: ensure the model generalizes, not just memorizes training data.  

---

### 🔹 Hold-out Split  
- Split dataset once into **train** and **test** sets.  
- Simple and fast, but performance may depend on which samples land in the test set.  

---

### 🔹 Train / Validation / Test  
- Often, we need a **validation set** to tune hyperparameters.  
- Typical ratios: 60% train / 20% validation / 20% test.  
- Test set is kept aside until the very end → avoids overfitting to validation.  

---

### 🔹 K-Fold Cross-Validation  
- Data is split into **k folds** (e.g., k=5).  
- Each fold is used once as test, the rest as train.  
- More robust than a single hold-out split, especially with small datasets.  
- Downside: more computation (train the model k times).  

---

⚠️ **Note:**  
- In industrial signal processing, datasets may be **limited** (few fault cases).  
- Cross-validation helps use data more effectively, while still testing generalization.  

## 📝 Exercises — Chapter 3: Data Preprocessing

*Hint: use fixed random seeds (e.g. 42) for any simulated noise/missing-data steps so results are reproducible for grading.*

These exercises focus on **signal cleaning, transformation, and preparation for ML**. Apply them on the Paderborn dataset and optionally on your own dataset.

1. **Handling Missing Values**
   - Simulate missing samples in a signal (e.g., remove ~1% for realism, or 5–10% to stress-test methods, using a fixed random seed).
   - Try different fill methods:
     - `ffill`, `bfill` (forward/backward fill in pandas).
     - Interpolation: `linear`, `polynomial`, `spline`, `time`.
     - Mean/median filling.
   - Compare the **frequency spectra (FFT/PSD)** of the original vs repaired signals.
   - Also inspect a **zoomed-in time window** — some spectral changes are subtle.

2. **Outlier Detection & Correction**
   - Detect outliers with both:
     - **IQR method** (points below Q1 − 1.5·IQR or above Q3 + 1.5·IQR).
     - **Z-score method**.
   - Show a **small table**: counts of flagged points by each method, and their overlap.
   - Correct outliers:
     - Use **Winsorization/capping** (`np.clip` or `scipy.stats.mstats.winsorize`) for plausible extremes.
     - Remove only values that are clearly impossible or known-bad.
     - Or impute with median/mean/predicted values.
   - Plot **before/after** and zoom in on corrected regions to ensure you didn’t erase real fault signatures.
   - **Optional:** Define domain-specific thresholds, if possible.

3. **Noise Filtering**
   - Add **synthetic noise** (impulse or Gaussian) to a clean signal.
   - Apply **median filter** (kernel sizes 3, 5, 7) and **moving average filter**.
   - Plot **original, noisy, and cleaned signals**.
   - Compare results and discuss which method removes noise best while preserving sharp transitions.

4. **Scaling & Transformations**
   - Add a log transform (np.log1p) and observe its effect on the signal histogram. If values are non-positive, shift the data slightly to make them positive before applying log1p. Compare before/after distributions.

   - **Optional:** Apply the same transformations to a **positively skewed dataset** (e.g., energy consumption values) and compare effects.
   - Reflect: When does PowerTransform provide meaningful changes? When does it behave like Standardization?

5. **Feature Extraction**
   - Extract features from a signal of a different class and compare features of two different classes (e.g., healthy vs faulty signals).
   - Extend the **Automation Challenge** from Chapter 2 to include **frequency-domain features** (dominant frequency, band power) in the feature matrix.  
   - Repeat the same visualization and exploration.  
   - If signals are long, extract features on **non-overlapping frames/windows**.

6. **Data Split**
   - **Train/Validation/Test Split:** Modify the existing hold-out code to create three subsets: 70% for training, 15% for validation, and 15% for testing. Print the shapes of each subset to confirm the split.

   - **Problem to solve (label mapping + encoding):** In the Paderborn dataset, `measurement.Label` does not exist. Students should:
      1. Assign the correct **class labels** by mapping folder names (e.g., `K001`, `K002`, …) to the dataset’s test-case/classes using the official Paderborn mapping (paper/website). Store the mapping in a `dict` or small CSV/JSON file.
      2. Apply **label encoding** to prepare labels for ML:
         - Use `sklearn.preprocessing.LabelEncoder` (or `pd.Categorical`) to convert class names → integer labels.
         - If needed for model input, use `OneHotEncoder` (or `pd.get_dummies`) to produce one-hot vectors.
      3. Verify correctness:
         - Print a few `(mat_file, inferred_label, encoded_label)` examples.
         - Print class counts: `np.unique(encoded_labels, return_counts=True)` and check for unexpected classes or imbalances.
      4. Use the (encoded) labels in your train/validation/test split.

   - **Challenge — Random Seeds:** Run your train/validation/test split multiple times with different `random_state` values and compare how the indices of samples in each subset change.

   - **K-Fold Cross-Validation Check:** After solving the label mapping and encoding problem, repeat the 5-fold (or k-fold) cross-validation from Chapter 3 and compare the results with your original cross-validation before label mapping. Discuss any differences in scores and why encoding affects the process.

7. **Project Dataset Processing**
   - Apply all necessary preprocessing on your own industrial dataset.
   - Make sure it is **ready for feature extraction and ML in the next chapter**.

# 📘 Chapter 4: Classical Machine Learning Methods
---
We now apply ML models (e.g., SVM, Random Forest) to extracted features.

# 📘 Chapter 5: Deep Learning Architectures
---
We extend to deep learning (CNNs, RNNs, Transformers) using raw signals.